In [1]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_predict,StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tracking_uri = "http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000" 

In [3]:
df  = pd.read_csv("data/preprocessing_df.csv")
df

,Unnamed: 0,text,category,word_count,num_chars,num_punctuation_chars
0,0,family mormon never tried explain still stare ...,1,39,259,0
1,1,buddhism much lot compatible christianity espe...,1,196,1268,0
2,2,seriously say thing first get complex explain ...,-1,86,459,0
3,3,learned want teach different focus goal wrappi...,0,29,167,0
4,4,benefit may want read living buddha living chr...,1,112,690,0
...,...,...,...,...,...,...
199697,162975,crore paid neerav modi recovered congress lead...,-1,18,107,0
199698,162976,dear rss terrorist payal gawar modi killing pl...,-1,36,248,0
199699,162977,cover interaction forum left,0,9,50,0
199700,162978,big project came india modi dream project happ...,0,13,77,0


In [4]:
df.text = df.text.fillna("")

In [5]:
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Exp3 : TF-IDF Trigram max_features")

2025/06/28 17:05:09 INFO mlflow.tracking.fluent: Experiment with name 'Exp3 : TF-IDF Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-114/489182401251117763', creation_time=1751119509167, experiment_id='489182401251117763', last_update_time=1751119509167, lifecycle_stage='active', name='Exp3 : TF-IDF Trigram max_features', tags={}>

In [6]:
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X = df.text
    y = df.category
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=114, stratify=y)

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators,class_weight="balanced", max_depth=max_depth, random_state=114)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/06/28 17:05:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 17:05:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763/runs/1d651953baf84faeb6e29e88a13890c8
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763


2025/06/28 17:06:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 17:06:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763/runs/664c5c2c82a1465ba51cb08300e5780e
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763


2025/06/28 17:07:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 17:07:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763/runs/968255cea3734d6e8b516902eb7ed4bb
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763


2025/06/28 17:12:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 17:12:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763/runs/e545f01ad6a945b9bf681583d368684a
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763


2025/06/28 17:13:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 17:13:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763/runs/459fc56607954bea96096ca31a91cd38
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/489182401251117763
